# 🎥 RightBiz Auto Video Generator
This notebook:
- Pulls your generated script
- Uses gTTS for AI voice
- Fetches stock video from Pexels
- Crops/resizes to vertical 1080x1920
- Adds captions
- Exports MP4 ready for TikTok/Instagram

In [ ]:
# ------------------ CONFIG ------------------
import os, requests
from gtts import gTTS
from moviepy.editor import VideoFileClip, AudioFileClip, TextClip, CompositeVideoClip

PEXELS_API_KEY = os.getenv('PEXELS_API_KEY') # set in Colab environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') # set in Colab environment variables (for Whisper/transcription)
OUTPUT_FILE = '/content/output.mp4'

# Example: Replace this with code that pulls the Generated Script from your Google Sheet via API
SCRIPT_TEXT = "Here’s how you can launch in 7 days with AI tools…"


# ------------------ TTS ------------------
print('Generating TTS...')
tts = gTTS(SCRIPT_TEXT)
tts.save('voice.mp3')


# ------------------ PEXELS VIDEO ------------------
print('Fetching stock video from Pexels...')
headers = {"Authorization": PEXELS_API_KEY}
resp = requests.get('https://api.pexels.com/videos/search?query=business&per_page=1', headers=headers)
resp.raise_for_status()
videos = resp.json().get('videos', [])
if not videos:
    raise SystemExit('No videos found from Pexels')
video_url = videos[0]['video_files'][0]['link']

with requests.get(video_url, stream=True) as r:
    r.raise_for_status()
    with open('background.mp4', 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)


# ------------------ TRANSFORM + COMPOSE ------------------
print('Composing final video...')
video = VideoFileClip('background.mp4').subclip(0,30)

# ensure vertical: center crop/resize to 1080x1920
video = video.resize(width=1080)
if video.h < 1920:
    video = video.on_color(size=(1080,1920), color=(0,0,0), pos=('center','center'))
else:
    video = video.crop(width=1080, height=1920, x_center=video.w/2, y_center=video.h/2)

audio = AudioFileClip('voice.mp3')
video = video.set_audio(audio)

# Simple caption
caption = SCRIPT_TEXT
caption_clip = TextClip(caption, fontsize=48, color='white', bg_color='black', size=(1000,200), method='caption')
caption_clip = caption_clip.set_position(('center', 1700)).set_duration(video.duration)

final = CompositeVideoClip([video, caption_clip])
final.write_videofile(OUTPUT_FILE, codec='libx264', audio_codec='aac', fps=24)

print('✅ Video generated ->', OUTPUT_FILE)
